## LP NOTES - comments/explanations in code

1. It's good practice to intersperse the code in your notebook with text that explains what you're doing (or, even better, *why* you're doing it). That could start with a title/header/intro that gives a couple of sentences of background. You've done this to a degree with comments in the code itself - which is good - but the big advantage of iPython notebooks (other than their interactivity and usefulness for experimentatation), is the ability to mix code and text in a natural way. I learned from experience that it can be difficult to read your code in six months' time (or the next day, sometimes), so I encourage you to take advantage of extensive notewriting to help your future self!

<div class="alert alert-error">
alert
</div>

In [ ]:
#View the content of the folder where all Dickeya genomes are stored
!ls

## LP NOTES - `import` statements

2. It's usual to put imports at the start of a module. You've got more flexibility in a notebook, but I find it handy (and readable) to keep all my imports together in a single cell. It can help to put a note or a comment line in there, as well.
3. You only need to import each module once. In fact, issuing `import <module name>` won't do anything the second time you use it. It *is* possible to reload/reimport a module, but typically you don't need to.

In [ ]:
import subprocess
import os  # LP: added here to enable a demo below

## LP NOTES - paths to files

4. In the code below, you've supplied a *hardcoded* absolute path to the same file three times. This can lead to problems:
    * even when copy/pasting, errors can creep in - one or more of those filenames can end up being incorrect, leading to bugs
    * if you need to change the filename at any point, you have to update it in three places - it would be easier if you only had to do it once
    * your absolute path is only ever likely to work on your machine, and if you don't move the directory/files - if you give this code to someone else (e.g. me ;) ) I can't run it. It would be more flexible and reproducible to use relative paths, with all necessary files available without reference to a unique absolute path from the root directory
    * the location is a location, but there's no direct information on why we are using it - putting the path information into a variable allows you to name it informatively, improving readability of your code.
These three options all do the same thing (on my machine):
```
# Hardcoded absolute path: will not work on your laptop, or if I move this directory
files = os.listdir("/Users/lpritc/Documents/JHI_Work/IBioIC/GitHub/IBioIC_enzymes/Identification_RBBH_2")
```
```
# Relative path: will work on my laptop, your laptop, and anyone else's who clones the repository
files = os.listdir(".")
```
```
# Relative path with variable name: the relative path will work anywhere, but
# using a variable means that you can define this in a single place and reuse it -
# avoiding copy/paste errors and bugs due to forgetting to change it in your code.
# Also, if the variable has a meaningful name, this can help readability of your
# code, and remind you why you're interested in that location.
dirname = "."
files = os.listdir(dirname)
```
In this context, splitting out the path into a variable is *defensive* programming (defending *you* against bugs creeping in by accident), and it's a way of programming that is worth making a habit.
5. The `i.endswith()` function works, but the `os.path.splitext()[-1]` alternative would be worth getting used to. It's more explicit (the `os.path` part implies you're dealing specifically with a file), and the function `os.path.splitext()` returns a tuple containing the filestem and the extension, which is often useful.

In [ ]:
# LP: Demonstrating os.path.splitext()
fname = "CSL_RW192.fasta"
os.path.splitext(fname)

In [ ]:
import os
for i in os.listdir("/Users/eirinixemantilotou/Documents/PhD/PhD_year1/Bioinformatics/Identifying_Dickeya_enzymes/Identification_RBBH_2"):
    if i.endswith(".fasta"):
        cmd = "makeblastdb -in %s -dbtype prot -title %s -out %s" % (i,i,i)
        cmd = cmd.split()
        subprocess.call(cmd)

In [ ]:
# import os  # LP: this line won't do anything

## LP NOTE: - coding style/conventions

6. The PEP8 style guide suggests using spaces after commas, and around operators (e.g. `%`). Which do you find easier to read?
    * `"blastp -query %s -db %s -out %s_%s.tab -outfmt 7" %(i,j,i.split('.')[0],j.split('.')[0])`
    * `"blastp -query %s -db %s -out %s_%s.tab -outfmt 7" % (i, j, i.split('.')[0], j.split('.')[0])`

In [ ]:
 for i in os.listdir("/Users/eirinixemantilotou/Documents/PhD/PhD_year1/Bioinformatics/Identifying_Dickeya_enzymes/Identification_RBBH_2"):
    if i.endswith(".fasta"):
        for j in os.listdir("/Users/eirinixemantilotou/Documents/PhD/PhD_year1/Bioinformatics/Identifying_Dickeya_enzymes/Identification_RBBH_2"):
            if j.endswith(".fasta"):
                if i != j:
                    cmd = "blastp -query %s -db %s -out %s_%s.tab -outfmt 7" %(i,j,i.split('.')[0],j.split('.')[0])
                    cmd = cmd.split()
                    print(cmd)
                    subprocess.call(cmd)

I have not run above the code below this cell as I had to wiat for the blastp. It has run the command for a couple of queries but there are more to go! 

In [ ]:
import csv

In [ ]:
fn_fwd = "blastp402_898_query_coverage.tab" # forward search
fn_rev = "blastp898_402_query_coverage.tab"  # reverse search

In [ ]:
# Read in forward data: query sequence name and target sequence
# name only (for now) for the best match.
# We define best match as the first match row for the query
# sequence.
def read_best_blast_hits(filename,pidthreshold,qcovsthreshold):
    """Read BLAST tab-separated output and return dictionary
    of best hits as {query1:subject1, query2:subject2...}"""
    results = {}  # empty dictionary holds results
    with open(filename, 'r') as fh:
        reader = csv.reader(fh, delimiter="\t")
        for row in reader:
            query_name = row[0]
            sbjct_name = row[1]
            pid = row[2]
            qcovs =row[12]
            # if query_name isn't in results dictionary, assume
            # this line is the top hit
            if query_name not in results:
                if float(pid) >= pidthreshold:
                    if float(qcovs)>= qcovsthreshold:
                        results[query_name] = sbjct_name
    return results

In [ ]:
# Get best hits in forward and reverse directions
fwd = read_best_blast_hits(fn_fwd ,80 ,80)
rev = read_best_blast_hits(fn_rev ,80 ,80)

In [ ]:
# Peek at results dictionaries
print(list(fwd.items())[:5])
print(list(rev.items())[:5])

In [ ]:
qf1 = fwd.keys()
print(qf1) # query name

In [ ]:
# Identify reciprocal best hits from forward and reverse
# best hit dictionaries
def calculate_rbbh(fwd, rev):
    """Returns a list of (query, subject) tuples that are
    reciprocal best hits, as defined from the passed pair
    of dictionaries:
    
    - fwd - best hits in the forward direction
    - rev - best hits in the reverse direction
    """
    rbbh = []
    for query_name in fwd.keys():
        match = fwd[query_name]
        if match in rev:
            if rev[match] == query_name:
                rbbh.append((query_name, fwd[query_name]))
    return rbbh

In [ ]:
# RBBH in forward direction
rbbh = calculate_rbbh(fwd, rev)
print(len(rbbh))

In [ ]:
# RBBH in reverse direction (should be same as forward)
rbbh_rev = calculate_rbbh(rev, fwd)
print(len(rbbh_rev))

In [ ]:
print(rbbh[:5])
print(rbbh_rev[:5]

In [ ]:
fwd = read_best_blast_hits(fn_fwd ,80 ,80)
rev = read_best_blast_hits(fn_rev ,80 ,80)
rbbh = calculate_rbbh(fwd, rev)

In [ ]:
print(len(rbbh))
print(rbbh[:20])